In [1]:
import psycopg2
import json
import pandas as pd
import requests

/Users/deel/Desktop/work/virtual_environments/payment_reconciliation_env/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [4]:
with open('/Users/deel/.Credentials/login_details.json', 'r') as fp:
    credentials = json.load(fp)

In [7]:
con = psycopg2.connect(dbname=credentials['database'], user=credentials['username'], password=credentials['password'], host=credentials['host'])

In [ ]:
# recommender = get_invoice_recommendations_locally(143103)

In [8]:
last_2_months_query = """select i.invoice_id, i.payments_transaction_id 
from "public".invoice_payments_transactions as i JOIN payments."Transaction"as t 
on i.payments_transaction_id=t.id
where t.created_at>'03/01/2022'"""

In [10]:
last_2_months_df = pd.read_sql(last_2_months_query, con)

In [12]:
last_2_months_df.shape

(12027, 2)

In [13]:
last_2_months_df.head(5)


,invoice_id,payments_transaction_id
0,2484631,6887851
1,2473682,6984974
2,2472973,6984967
3,2484657,6995201
4,2453291,6992951


In [20]:
from time import time

In [22]:
time()

1651502110.920141

In [ ]:
start = time()
url = 'http://127.0.0.1:5000//get-transaction-invoice-comparison-locally'
correct_matches = []
faulty = []
for i, row in last_2_months_df[2000:].iterrows():
#     if i>1500:
#         break
    if not i%100:
        print(f"time elapsed to {i}: {time()-start}")
    if not i%250:
        df = pd.DataFrame(correct_matches)
        df.to_csv(f"correct_matches_{i-250}-{i}.csv")
        correct_matches = []
    payload = {'transactionId': str(row['payments_transaction_id']), 'invoiceId': str(row['invoice_id'])}
    response = requests.post(url, json = payload).json()
    if response:
        correct_matches.append(response[0])
    else:
        faulty.append(payload)
stop = time()
print(f"runtime: {stop-start}")

time elapsed to 2000: 0.0006580352783203125
time elapsed to 2100: 197.59258604049683
time elapsed to 2200: 399.7342939376831
time elapsed to 2300: 600.8668229579926
time elapsed to 2400: 802.5833978652954
time elapsed to 2500: 1008.8035538196564
time elapsed to 2600: 1211.768759727478
time elapsed to 2700: 1415.752091884613
time elapsed to 2800: 1625.4618499279022
time elapsed to 2900: 1827.3266439437866
time elapsed to 3000: 2030.613208770752
time elapsed to 3100: 2232.514745950699
time elapsed to 3200: 2438.652003765106
time elapsed to 3300: 2645.1570320129395
time elapsed to 3400: 2849.37305188179
time elapsed to 3500: 3058.162905931473
time elapsed to 3600: 3265.4197788238525
time elapsed to 3700: 3470.530194044113
time elapsed to 3800: 3681.987808942795
time elapsed to 3900: 3896.3955039978027
time elapsed to 4000: 4101.638688087463
time elapsed to 4100: 4314.728980064392
time elapsed to 4200: 4525.682002067566
time elapsed to 4300: 4735.995548963547
time elapsed to 4400: 4949.520

In [ ]:
len(correct_matches)

In [54]:
len(faulty)

9

In [ ]:
correct_matches_df = pd.DataFrame(all_correct_matches)

In [ ]:
correct_matches_df['match_score'].sum()

In [ ]:
correct_matches_df['deel_ref_match'].sum()

In [55]:
correct_matches_df.to_csv('correct_matches_0-1000.csv')

In [10]:
matched_invoice_transaction_df = pd.read_sql("select * from public.invoice_payments_transactions", con)

In [18]:
transaction_id = 7443022
transactions_query = f"""
        SELECT
            *
        FROM
            payments. "Transaction"
        WHERE
            id in ({str(transaction_id)})"""

transaction_df = pd.read_sql(transactions_query, con)

In [19]:
transaction_df

,id,provider,provider_ref,type,account,amount,currency,description,data,created_at
0,7443022,wise,TRANSFER-374619443,INCOMING,3714250,9407.3,EUR,Received money from CATENA OPERATIONS LIMITED ...,"{'date': '2022-03-04T11:39:09.595142Z', 'type'...",2022-03-04 11:39:09.595000+00:00


In [20]:
transaction_df.iloc[0]['data']

{'date': '2022-03-04T11:39:09.595142Z',
 'type': 'CREDIT',
 'amount': {'zero': False, 'value': 9407.3, 'currency': 'EUR'},
 'details': {'type': 'DEPOSIT',
  'senderName': 'CATENA OPERATIONS LIMITED',
  'description': 'Received money from CATENA OPERATIONS LIMITED with reference Relation: Professional ServiceReason: Payment for ServicesCatena StatementS 2022 60012 Ref noDEELSKXHW8',
  'senderAccount': '(VALLMTMTXXX) MT22VALL22013000000040022643780',
  'paymentReference': 'Relation: Professional ServiceReason: Payment for ServicesCatena StatementS 2022 60012 Ref noDEELSKXHW8'},
 'accountId': 3714250,
 'totalFees': {'zero': True, 'value': 0, 'currency': 'EUR'},
 'attachment': None,
 'runningBalance': {'zero': False, 'value': 17030968.15, 'currency': 'EUR'},
 'exchangeDetails': None,
 'referenceNumber': 'TRANSFER-374619443',
 'activityAssetAttributions': []}

In [21]:
invoice_id = 2469750
invoice_query = f"""
SELECT * 
FROM invoice
WHERE id IN ({str(invoice_id)})"""

In [22]:
invoice_df = pd.read_sql(invoice_query, con)

In [23]:
invoice_df

,created_at,updated_at,id,amount,invoice_id,contract_id,status,user_id,internal_tx_id,payment_method,...,autopilot_root_invoice_id,refund_for,source_of_revenue,moved_to_processing_by_user_id,skip_releasing_balance,released_by_date,autopay_rule_id,sender_legal_entity_id,is_sealed,proof_of_payment_file_id
0,2022-02-28 07:37:12.768,2022-03-07 19:21:21.635,2469750,9214.04,7eS1awDLlhIPsBKm_jQLl,-200,paid,123005,20914788,bank_transfer,...,None,None,INVOICE,99044,False,None,None,None,False,None


In [24]:
for i, row in invoice_df.iterrows():
    for col in invoice_df.columns:
        print(col)
        print(row[col])
        print('************')

created_at
2022-02-28 07:37:12.768000
************
updated_at
2022-03-07 19:21:21.635000
************
id
2469750
************
amount
9214.04
************
invoice_id
7eS1awDLlhIPsBKm_jQLl
************
contract_id
-200
************
status
paid
************
user_id
123005
************
internal_tx_id
20914788
************
payment_method
bank_transfer
************
tx_id
DEELSKXHW8
************
contractor_id
-100
************
rates
{'rates': {'USD': '1', 'USDC': 1, 'AED': '3.673096', 'ALL': '108.612556', 'AMD': '482.077401', 'ARS': '107.459109', 'AUD': '1.389439', 'AZN': '1.701083', 'BAM': '1.750849', 'BBD': '2.021681', 'BDT': '86.085088', 'BGN': '1.73754', 'BHD': '0.377046', 'BMD': '1', 'BND': '1.356298', 'BOB': '6.883658', 'BRL': '5.163565', 'BSD': '1.001262', 'BWP': '11.615574', 'BYN': '2.761279', 'BZD': '2.01828', 'CAD': '1.27775', 'CDF': '2013.999741', 'CHF': '0.92619', 'CLP': '810.879476', 'CNY': '6.310994', 'COP': '3919.61', 'CRC': '641.983708', 'CUP': '26.5', 'CVE': '98.700729', 'CZK

In [ ]:
transaction_df.iloc[0]['data']

In [ ]:
matched_invoice_transaction_df.head(10)

In [ ]:
transaction_df

In [ ]:
matched_invoice_transaction_df[matched_invoice_transaction_df['payments_transaction_id']==257382]

In [ ]:
transaction_id_list = [str(id) for id in matched_invoice_transaction_df['payments_transaction_id'][:20]]
transaction_string = ','.join(transaction_id_list)

In [ ]:
transactions = pd.read_sql(f'select * from payments. "Transaction" where id in ({5173879})', con)

In [ ]:
transactions

In [ ]:
invoices = pd.read_sql(f'select * from public.invoice where id in (2044669, 2044677, 2044683)', con)

In [ ]:
invoices


In [ ]:
invoices

In [ ]:
transaction = transactions.iloc[0]

In [ ]:
transaction['id']

In [ ]:
url="http://127.0.0.1:5000/get-examples-locally"


In [ ]:
transactions

In [ ]:
request_list = []
for i, transaction in transactions[:5].iterrows():

    payload = {'transactionId': str(transaction['id'])}
    response = requests.post(url, json=payload)
    request_list.append({'transaction': transaction['id'], 'response': response.json()})

In [ ]:
import requests
url="http://127.0.0.1:5000//get-invoice-recommendations-locally"
payload = {'transactionId': str(6597058)}
response = requests.post(url, json=payload)

In [ ]:
response.json()

In [ ]:
match_df = pd.read_sql("select * from invoice_payments_transactions where payments_transaction_id=6403097", con)

In [ ]:
match_df

In [ ]:
response_df = pd.DataFrame(response.json())

In [ ]:
response_df

In [ ]:
request_list[2]['response']

In [ ]:
for d in request_list[4]['response']:
    print(d['match_score'])

In [ ]:
for response  in request_list:
    print response['transactionId']
    print
    

In [ ]:
matched_invoice_transaction_df

In [ ]:
pd.read_sql(f'select * from payments. "Transaction" where id = 3031744', con)['data']

In [ ]:
pd.read_sql(f'select * from public. "invoice" where id = 1687151', con)

In [ ]:
transaction_3031744 = pd.read_sql('select * from payments. "Transaction" where id=3031744', con)

In [ ]:
transaction_3031744

In [ ]:
invoice_refs = pd.read_sql('select ref from public.invoice', con)

In [ ]:
def lower_str(x):
    if type(x)==type('hi'):
        return x.lower()
    return ''

In [ ]:
invoice_refs['lower_refs'] = invoice_refs['ref'].apply(lower_str)

In [ ]:
invoice_refs[invoice_refs['lower_refs']==str.lower('DEELMRWSF4')]

In [ ]:
str.lower('DEELMRWSF4')

In [ ]:
transaction_3031744.iloc[0].data

## investigating a particular transaction

In [10]:
payload_2 = {'transactionId': 122508}
url_2 = 'http://127.0.0.1:5000//get-invoice-recommendations-locally'
response_2 = requests.post(url_2, json = payload_2)

In [11]:
response_2.json()

[]

In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame(response_2.json())

In [ ]:
for recommendations we weren't pulling transfers made via coinbase, so all USDC payments were being skipped. Added coinbase to transfer methods in query

In [ ]:
get-transaction-invoice-comparison-locally
def compare_transaction_invoice_locally():
    data = request.json
    transaction_id = data['transactionId']
    invoice_id = data['invoiceId']

In [45]:
url = 'http://127.0.0.1:5000//get-transaction-invoice-comparison-locally'
payload = {'transactionId': 12761016, 'invoiceId': 3047744 }

In [50]:
response = requests.post(url, json = payload)

In [51]:
response.json()

[{'amount_difference': -13.80999999999949,
  'company_name_match_ratio': 1.0,
  'days_until_money_received': 0,
  'deel_ref_extracted': True,
  'deel_ref_match': True,
  'id': 3047744,
  'match_score': 0.9996544477569714,
  'rel_abs_amount_difference': 0.0018429452961523,
  'rel_amount_difference': -0.0018429452961523,
  'transaction_amount': 7479.63,
  'transaction_deel_reference': 'DEELESJ5KN'}]

In [13]:
transactions_query = f"""
            SELECT
                *
            FROM
                payments. "Transaction"
            WHERE
                id = 122508"""

In [19]:
transaction = pd.read_sql(transactions_query, con)

In [21]:
transaction.columns

Index(['id', 'provider', 'provider_ref', 'type', 'account', 'amount',
       'currency', 'description', 'data', 'created_at'],
      dtype='object')

In [22]:
transaction['data']

0    {'amount': 515, 'account': {'aba': '322271627'...
Name: data, dtype: object